In [1]:
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
import sys
sys.path.append('.')
from src.client_tools import get_location
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('.')
from src.utils import step_printer
from termcolor import cprint

from rich import print

In [ ]:
base_url = os.getenv("REMOTE_BASE_URL")

client = LlamaStackClient(
    base_url=base_url
)

# model_id will later be used to pass the name of the desired inference model to Llama Stack Agents/Inference APIs
model_id = "deepseek-r1-0528-qwen3-8b-bnb-4bit" # "granite-3.2-2b-instruct" # /mnt/models/DeepSeek-R1-0528-Qwen3-8B-Q4_K_M.gguf" # granite-3.2-2b-instruct

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Inference Parameters:
        Model: granite-3.2-2b-instruct
        Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 15000}
        stream: True

In [7]:
agent = ReActAgent(
            client=client,
            model=model_id,
            tools=["mcp::github", "mcp::openshift"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
        )

user_prompts = ["""Get file content using GitHub MCP owner=eformat, repo=agentic-llamastack, path=notebooks/prompts/namespace-create.md, ref=main
                Use the namespace_create tool to create the namespace listed in the file one at a time.
                """
                ]

session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 

==================================================

Processing user query: Get file content using GitHub MCP owner=eformat, repo=agentic-llamastack, path=notebooks/prompts/namespace-create.md, ref=main
                Use the namespace_create tool to create the namespace listed in the file one at a time.
                


==================================================

inference> {
    "thought": "I need to first fetch the content of the specified file from the GitHub repository using the 'get_file_contents' tool. Then, I will parse the content to extract the namespace creation command.",
    "action": {
        "tool_name": "get_file_contents",
        "tool_params": [
            {
                "name": "owner",
                "value": "eformat"
            },
            {
                "name": "repo",
                "value": "agentic-llamastack"
            },
            {
                "name": "path",
                "value": "notebooks/prompts/namespace-create.md"
            },
            {
                "name": "branch",
                "value": "main"
            }
        ]
    },
    "answer": null
}
tool_execution> Tool:get_file_contents Response:{"type":"text","text":"{\n  \"name\": \"namespace-create.md\",\n  \"path\": \"notebooks/prompts/namespace-create.md\",\n  \"sha\": \"d5c84f95976fa5f809bede182c6d4ac3d35a5d72\",\n  \"s

In [8]:
user_prompts = ["""Get file content using GitHub MCP owner=eformat, repo=agentic-llamastack, path=notebooks/prompts/namespace-create.md, ref=main
                Use the namespace_delete tool to delete the namespace listed in the file one at a time.
                """
                ]

session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 

==================================================

Processing user query: Get file content using GitHub MCP owner=eformat, repo=agentic-llamastack, path=notebooks/prompts/namespace-create.md, ref=main
                Use the namespace_delete tool to delete the namespace listed in the file one at a time.
                


==================================================

inference> {
    "thought": "I need to first fetch the content of the specified file from the GitHub repository. Then, I will parse the content to extract the namespace deletion command. After that, I will use the `namespace_delete` tool to execute each command one by one.",
    "action": {
        "tool_name": "get_file_contents",
        "tool_params": [
            {
                "name": "owner",
                "value": "eformat"
            },
            {
                "name": "repo",
                "value": "agentic-llamastack"
            },
            {
                "name": "path",
                "value": "notebooks/prompts/namespace-create.md"
            },
            {
                "name": "ref",
                "value": "main"
            }
        ]
    },
    "answer": null
}
tool_execution> Tool:get_file_contents Response:{"type":"text","text":"{\n  \"name\": \"namespace-create.md\",\n  \"path\": \"notebooks/prompts/namespace-create.md\",\n  \"sha\": \"d